## Описание проекта
Аналитики крупного интернет-магазина вместе с отделом маркетинга подготовили список гипотез для увеличения выручки.  
Необходимо приоритизировать гипотезы, запустить A/B-тест и проанализировать результаты. 

## Данные
1. Файл `/datasets/hypothesis.csv`  
`Hypothesis` — краткое описание гипотезы;  
`Reach` — охват пользователей по 10-балльной шкале;  
`Impact` — влияние на пользователей по 10-балльной шкале;  
`Confidence` — уверенность в гипотезе по 10-балльной шкале;  
`Efforts` — затраты ресурсов на проверку гипотезы по 10-балльной шкале. Чем больше значение Efforts, тем дороже проверка гипотезы.  
2. Файл `/datasets/orders.csv`  
`transactionId` — идентификатор заказа;  
`visitorId` — идентификатор пользователя, совершившего заказ;  
`date` — дата, когда был совершён заказ;  
`revenue` — выручка заказа;  
`group` — группа A/B-теста, в которую попал заказ.  
3. Файл `/datasets/visitors.csv`  
`date` — дата;  
`group` — группа A/B-теста;  
`visitors` — количество пользователей в указанную дату в указанной группе A/B-теста.

## Загружаем необходимые библиотеки

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
import warnings
import scipy.stats as stats
# конвертеры, которые позволяют использовать типы pandas в matplotlib  
register_matplotlib_converters()

## Приоритизация гипотез

In [2]:
# запишем датафрейм
hypothesis = pd.read_csv('/datasets/hypothesis.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/hypothesis.csv'

In [ ]:
# смотрим на содержимое датафрейма
pd.set_option('max_colwidth', 200) # добавляю
display(hypothesis)

<div class="alert alert-block alert-warning">
    

**Комментарий от ревьюера**

Удобнее вывести названия гипотез целиком: `pd.set_option('max_colwidth', 200)` и округлить значения в колонке ICE до 2 знаков после запятой.
    

</div>

<div class="alert alert-info">
Исправляю
</div>

<div class="alert alert-success">
<b> Комментарий от ревьюера 2</b>

Молодец, что отредактировал таблицу, теперь она выглядит лучше. 

</div>

In [ ]:
# применим фреймворк ICE для приоритизации гипотез, отсортируем их по убыванию приоритета
hypothesis['ICE'] = (hypothesis['Impact'] * hypothesis['Confidence'] / hypothesis['Efforts']).round(2) # добавил .round(2)
display(hypothesis[['Hypothesis', 'ICE']].sort_values(by = 'ICE', ascending = False))

In [ ]:
# Применим фреймворк RICE для приоритизации гипотез, Отсортируем их по убыванию приоритета.
hypothesis['RICE'] = hypothesis['Reach'] * hypothesis['Impact'] * hypothesis['Confidence'] / hypothesis['Efforts']
display(hypothesis[['Hypothesis', 'RICE']].sort_values(by = 'RICE', ascending = False))

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**

Молодец, что выводишь таблицу дважды, каждый раз с сортировкой по одному из фреймворков - так различия в их работе максимально наглядны.
</div>

**Вывод:**  
Самый высокий ICE у 8, 0 и 7 гипотезы.  
Самый высокий RICE у 7, 2, 0 и 6 гипотезы.  
Изменение приоритизации по RICE связано с максимальным значением Reach (затрагиваемый охват пользователей) у 7 гипотезы.  
В нашем случае приоритетными гипотезами будут 7, 2, 0.

<div class="alert alert-block alert-warning">
    

**Комментарий от ревьюера**

Стоит привести названия гипотез полностью или использовать нумерацию, указанную в таблице (либо изменить ее, чтобы она начиналась не с нуля). Также здесь можно выбрать гипотезу, которую мы будем тестировать первой, чтобы между частями проекта была логическая связь.

</div>

## Анализ A/B-теста
A/B тест проведен, необходимо проанализировать результаты

In [ ]:
# записываем получившиеся датафреймы
orders = pd.read_csv('/datasets/orders.csv')
visitors = pd.read_csv('/datasets/visitors.csv')

In [ ]:
# смотрим на получившиеся датафреймы
display(orders)
orders.info()

In [ ]:
# смотрим на получившиеся датафреймы
display(visitors)
visitors.info()

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера**
При знакомстве с данными нам нужно еще выполнить проверку на дубликаты.
</div>

<div class="alert alert-info">
Исправляю
</div>

<div class="alert alert-success">
<b> Комментарий от ревьюера 2</b>

Здорово, теперь все необходимые проверки у нас есть.

</div>

In [ ]:
# проверяем на наличие явные дубликатов
print('Явных дубликатов по orders:', orders.duplicated().sum())
print('Явных дубликатов по visitors:', visitors.duplicated().sum())

<div class="alert alert-block alert-warning">
    
**Комментарий от ревьюера**

Полезно также выполнить проверку, не попадают ли какие-то пользователи в обе группы - по таблице с заказами.

</div>

In [ ]:
# приводим в формату дата по столбцам date обоих датафреймов
orders['date'] = pd.to_datetime(orders['date'])
visitors['date'] = pd.to_datetime(visitors['date'])

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**

Да, здесь нужно поменять формат даты.
</div>

In [ ]:
# создаем массив уникальных пар значений дат и групп теста
datesGroups = orders[['date','group']].drop_duplicates()
# получаем агрегированные кумулятивные по дням данные о заказах 
ordersAggregated = datesGroups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'transactionId' : 'nunique', 'visitorId' : 'nunique', 'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])
# получаем агрегированные кумулятивные по дням данные о посетителях интернет-магазина 
visitorsAggregated = datesGroups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'visitors' : 'sum'}), axis=1).sort_values(by=['date','group'])
# объединяем кумулятивные данные в одной таблице и присваиваем ее столбцам понятные названия
cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

display(cumulativeData)

<div class="alert alert-block alert-warning">

**Комментарий от ревьюера**

Здесь можно проверить, корректно ли был создан датафрейм cumulativeData. Например, совпадают ли минимальная и максимальная даты в этом датафрейме с минимальной и максимальной датой в исходных данных.

</div>

In [ ]:
# строим график кумулятивной выручки по дням и группам

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]
# оформляем график
plt.figure(figsize=(16,6)) #добавил
plt.title("График кумулятивной выручки по дням и группам") #добавил
plt.xlabel("Дата") #добавил
plt.ylabel("Выручка") #добавил
plt.grid() #добавил
# Строим график выручки группы А
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='A')
# Строим график выручки группы B
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='B')
#plt.xticks(rotation=90)
plt.legend()
plt.show()

<div class="alert alert-block alert-danger">
    

**Комментарий от ревьюера**

Стоит немного увеличить размер графика (чтобы для дат было достаточно места), добавить название графика и названия осей. Этот комментарий относится ко всем графикам в проекте. 
</div>

<div class="alert alert-info">
Исправляю
</div>

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера 2**

Теперь графики оформлены очень аккуратно: есть названия, подписи осей, настроен размер 👍
</div>

**Вывод:**  
Выручка по группе В в середине теста (2019-08-17) резко вырастает и продолжает расти значительно превышая выручку по группе А.  
Это может свидетельствовать или о резком росте числа заказов, или об увеличении среднего чека заказов.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**

Верно, пока мы не знаем, с чем связан скачок в группе B: с резким увеличением количества заказов или с ростом стоимости.
</div>

In [ ]:
# построим график кумулятивного среднего чека по группам
# оформляем график
plt.figure(figsize=(16,6)) #добавил
plt.title("График кумулятивного среднего чека по группам") #добавил
plt.xlabel("Дата") #добавил
plt.ylabel("Средний чек") #добавил
plt.grid() #добавил
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue']/cumulativeRevenueA['orders'], label='A')
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue']/cumulativeRevenueB['orders'], label='B')
#plt.xticks(rotation=90)
plt.legend()
plt.show()

**Вывод:**  
Снова виден резкий всплеск 2019-08-17 и затем медленное снижение среднего чека по группе В. Вероятно был крупный заказ на эту дату.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**
    
Видим резкий всплеск на графике, возможно, в середине теста в группу B попали крупные заказы.
</div>

In [ ]:
# строим график относительного изменения кумулятивного среднего чека группы B к группе A
# собираем данные в одном датафрейме
mergedCumulativeRevenue = cumulativeRevenueA.merge(cumulativeRevenueB, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])
# оформляем график
plt.figure(figsize=(16,6)) #добавил
plt.title("График относительного изменения кумулятивного среднего чека группы B к группе A") #добавил
plt.xlabel("Дата") #добавил
plt.ylabel("Отношение среднего чека") #добавил
plt.grid() #добавил
# cтроим отношение средних чеков
plt.plot(mergedCumulativeRevenue['date'], (mergedCumulativeRevenue['revenueB']/mergedCumulativeRevenue['ordersB'])/(mergedCumulativeRevenue['revenueA']/mergedCumulativeRevenue['ordersA'])-1)
# добавляем ось X
plt.axhline(y=0, color='black', linestyle='--') 
#plt.xticks(rotation=90)
plt.show()

**Вывод:**  
Наблюдаются постоянные резкие скачки среднего чека. Вероятно были крупные заказы по группе В.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**
    
Действительно, этот график не стабилизировался.
</div>

In [ ]:
# строим график кумулятивного среднего количества заказов на посетителя по группам
# датафрейм с кумулятивным средним количеством заказов на посетителя по группе А
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','visitors', 'orders']]
# датафрейм с кумулятивным средним количеством заказов на посетителя по группе В
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','visitors', 'orders']]
# оформляем график
plt.figure(figsize=(16,6)) #добавил
plt.title("График кумулятивного среднего количества заказов на посетителя по группам") #добавил
plt.xlabel("Дата") #добавил
plt.ylabel("Среднее количество заказов на посетителя") #добавил
plt.grid() #добавил
# строим график среднего количества заказов по посетителям по группе А
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['orders']/cumulativeRevenueA['visitors'], label='A')
# строим график среднего количества заказов по посетителям по группе В
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['orders']/cumulativeRevenueB['visitors'], label='B')
#plt.xticks(rotation=90)
plt.legend()
plt.show()

**Вывод:**  
По группе В среднее количество заказов на посетителей превышает среднее количество заказов по группе А.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**
    
Да, группа В выигрывает на протяжении всего теста.
</div>

In [ ]:
# строим график относительного изменения кумулятивного среднего количества заказов на посетителя группы B к группе A
# собираем данные в одном датафрейме
mergedCumulativeRevenue = cumulativeRevenueA.merge(cumulativeRevenueB, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])
# оформляем график
plt.figure(figsize=(16,6)) #добавил
plt.title("График относительного изменения кумулятивного среднего количества заказов на посетителя группы B к группе A") #добавил
plt.xlabel("Дата") #добавил
plt.ylabel("Отношение среднего количества заказов") #добавил
plt.grid() #добавил
# cтроим отношение среднего количества заказов
plt.plot(mergedCumulativeRevenue['date'], (mergedCumulativeRevenue['ordersB']/mergedCumulativeRevenue['visitorsB'])/(mergedCumulativeRevenue['ordersA']/mergedCumulativeRevenue['visitorsA'])-1)
# добавляем ось X
plt.axhline(y=0, color='black', linestyle='--') 
plt.axhline(y=0.15, color='black', linestyle='--') #добавил
#plt.xticks(rotation=90)
plt.show()

<div class="alert alert-block alert-warning">
    
**Комментарий от ревьюера**

Для удобства на график можно добавить еще одну линию серым цветом на значение 0.15 по оси y, так как около этого значения намечается тенденция к стабилизации.
    
</div>

<div class="alert alert-info">
Исправляю
</div>

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера 2**

👍
</div>

**Вывод:**  
Резкий всплекск наблюдается 05.08.2019, затем идет плавное снижение. Возможно выброс таким образом влияет на показатели.

In [ ]:
# строим точечный график количества заказов по пользователям.
# готовим таблицу для исследования по кол-ву заказов
ordersByUsers = orders.groupby('visitorId', as_index=False).agg({'transactionId': 'nunique'})
# переименовываем столбцы
ordersByUsers.columns = ['userId', 'orders']
display(ordersByUsers.sort_values(by='orders', ascending=False).head(10))
# серия из чисел от 0 до количества наблюдений в ordersByUsers
x_values = pd.Series(range(0, len(ordersByUsers)))
# оформляем график
plt.figure(figsize=(16,6)) #добавил
plt.title("График количества заказов по пользователям") #добавил
plt.xlabel("Количество пользователей") #добавил
plt.ylabel("Количество заказов") #добавил
plt.grid() #добавил
plt.scatter(x_values, ordersByUsers['orders'])
plt.show()

**Вывод:**  
Пользователей, заказавших 2 и более раза немного.

In [ ]:
# посчитаем 95-й и 99-й перцентили количества заказов на пользователя
display(np.percentile(ordersByUsers['orders'], [90, 95, 99]))

**Вывод:**  
Не более 5% посетителей делали 2 заказа, не более 1% делали более 4 заказов.

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера**

Здесь и для следующего вопроса надо добавить, какой перцентиль ты будешь считать границей выбросов.

</div>

<div class="alert alert-info">
Исправляю
</div>

**Принимаем, что выбросом будет считаться свыше двух заказов.**

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера 2**
    
Такое определение выбросов для количества заказов корректно.
</div>

In [ ]:
# построим точечный график стоимостей заказов
# готовим данные для анализа по стоимости заказа
transactionBySumm = orders.groupby('transactionId', as_index=False).agg({'revenue': 'sum'})
# переименовываем столбцы
transactionBySumm.columns = ['transactionId', 'revenue']
display(transactionBySumm.sort_values(by='revenue', ascending=False).head(10))
# серия из чисел от 0 до количества наблюдений в ordersByUsers
x_values = pd.Series(range(0, len(transactionBySumm)))
# оформляем график
plt.figure(figsize=(16,6)) #добавил
plt.title("График стоимостей заказов") #добавил
plt.xlabel("Количество пользователей") #добавил
plt.ylabel("Стоимость заказов") #добавил
plt.grid() #добавил
plt.scatter(x_values, transactionBySumm['revenue'])
plt.show()

**Вывод:**  
Явно просматривается аномалия - покупка на 1294500 и 202740.

<div class="alert alert-block alert-warning">
    
**Комментарий от ревьюера**

Полезно добавить еще один график, ограничив значения по оси Y. Чтобы двух явных выбросов не было видно, а мы могли рассмотреть основную часть заказов.

</div>

<div class="alert alert-info">
Исправляю
</div>

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера 2**
    
Удобно, когда можно рассмотреть и выбросы, и основную часть заказов.
</div>

In [ ]:
# исключаем выбросы, строим график заказов
# готовим данные для анализа по стоимости заказа
transactionBySummClear = transactionBySumm[transactionBySumm['revenue'] < 200000]
display(transactionBySummClear.sort_values(by='revenue', ascending=False).head(10))
# серия из чисел от 0 до количества наблюдений в ordersByUsers
x_values = pd.Series(range(0, len(transactionBySummClear)))
# оформляем график
plt.figure(figsize=(16,6)) #добавил
plt.title("График стоимостей заказов") #добавил
plt.xlabel("Количество пользователей") #добавил
plt.ylabel("Стоимость заказов") #добавил
plt.grid() #добавил
plt.scatter(x_values, transactionBySummClear['revenue'])
plt.show()

In [ ]:
# посчитаем 95-й и 99-й перцентили стоимости заказов
display(np.percentile(transactionBySumm['revenue'], [90, 95, 99]))

**Вывод:**  
Не более чем у 5% посетителей заказы стоят выше 28000 и только у 1% выше 58233.

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера 2**

Здесь тоже надо добавить, какой перцентиль ты будешь считать границей выбросов.

</div>

<div class="alert alert-info">
Исправляю
</div>

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера 3**
    
Отлично, теперь здесь есть все что нужно.
</div>

**Принимаем, что заказы выше 28000 считаются выбросом.**

<div class="alert alert-block alert-success"> <b>Комментарий от ревьюера:</b> Молодец, что разобрался с аномалиями в данных и корректно освоил расчет кумулятивных метрик.
</div>

--------------------------------

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера**
    
Чтобы расчеты было проще понять, перед каждым из 4 расчетов надо добавить вступление:

* сформулируй нулевую и альтернативную гипотезы;
* укажи, какое значение alpha ты будешь использовать, чтобы не искать его в коде;
* укажи, какой метод ты используешь для проверки и почему он подходит здесь лучше всего.
  
</div>

~~Проверим нулевую гипотезу: между группами А и В есть статистически значимое различие.  
Принимаем значение alpha = 0.05.  
Применим метод Уилкоксона-Манна-Уитни для двух выборок с вероятными выбросами.~~

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера 2**

Во вступления к расчетам надо добавить еще альтернативную гипотезу.
    
Чтобы в гипотезах была полная информация о каждом расчете, можно сформулировать их примерно так:
 
**Нулевая гипотеза** Статистически значимых различий в ... между группами по ... данным нет.

**Альтернативная гипотеза** Статистически значимые различия в ... между группами по ... данным есть.

</div>

<div class="alert alert-info">
Исправляю
</div>

Проверим **нулевую гипотезу**:  
Статистически значимых различий в среднем количестве заказов по посетителям между группами по "сырым" данным нет.  
и **альтернативную гипотезу**:  
Статистически значимые различия в среднем количестве заказов по посетителям между группами по "сырым" данным есть.  
Принимаем значение alpha = 0.05.  
Применим метод Уилкоксона-Манна-Уитни для двух выборок с вероятными выбросами.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера 3**

Хорошо получилось, так сразу понятна суть расчета.
</div>

In [ ]:
# посчитаем статистическую значимость различий в среднем количестве заказов на посетителя между группами по «сырым» данным
visitorsADaily = visitors[visitors['group'] == 'A'][['date', 'visitors']]
visitorsADaily.columns = ['date', 'visitorsPerDateA']

visitorsACummulative = visitorsADaily.apply(
    lambda x: visitorsADaily[visitorsADaily['date'] <= x['date']].agg(
        {'date': 'max', 'visitorsPerDateA': 'sum'}
    ),
    axis=1,
)
visitorsACummulative.columns = ['date', 'visitorsCummulativeA']

visitorsBDaily = visitors[visitors['group'] == 'B'][['date', 'visitors']]
visitorsBDaily.columns = ['date', 'visitorsPerDateB']

visitorsBCummulative = visitorsBDaily.apply(
    lambda x: visitorsBDaily[visitorsBDaily['date'] <= x['date']].agg(
        {'date': 'max', 'visitorsPerDateB': 'sum'}
    ),
    axis=1,
)
visitorsBCummulative.columns = ['date', 'visitorsCummulativeB']

ordersADaily = (
    orders[orders['group'] == 'A'][['date', 'transactionId', 'visitorId', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transactionId': pd.Series.nunique, 'revenue': 'sum'})
)
ordersADaily.columns = ['date', 'ordersPerDateA', 'revenuePerDateA']

ordersACummulative = ordersADaily.apply(
    lambda x: ordersADaily[ordersADaily['date'] <= x['date']].agg(
        {'date': 'max', 'ordersPerDateA': 'sum', 'revenuePerDateA': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])
ordersACummulative.columns = [
    'date',
    'ordersCummulativeA',
    'revenueCummulativeA',
]

ordersBDaily = (
    orders[orders['group'] == 'B'][['date', 'transactionId', 'visitorId', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transactionId': pd.Series.nunique, 'revenue': 'sum'})
)
ordersBDaily.columns = ['date', 'ordersPerDateB', 'revenuePerDateB']

ordersBCummulative = ordersBDaily.apply(
    lambda x: ordersBDaily[ordersBDaily['date'] <= x['date']].agg(
        {'date': 'max', 'ordersPerDateB': 'sum', 'revenuePerDateB': 'sum'}
    ),
    axis=1,
).sort_values(by=['date'])
ordersBCummulative.columns = [
    'date',
    'ordersCummulativeB',
    'revenueCummulativeB',
]

data = (
    ordersADaily.merge(
        ordersBDaily, left_on='date', right_on='date', how='left'
    )
    .merge(ordersACummulative, left_on='date', right_on='date', how='left')
    .merge(ordersBCummulative, left_on='date', right_on='date', how='left')
    .merge(visitorsADaily, left_on='date', right_on='date', how='left')
    .merge(visitorsBDaily, left_on='date', right_on='date', how='left')
    .merge(visitorsACummulative, left_on='date', right_on='date', how='left')
    .merge(visitorsBCummulative, left_on='date', right_on='date', how='left')
)

#print(data.head(5))

ordersByUsersA = (
    orders[orders['group'] == 'A']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersA.columns = ['visitorId', 'orders']

ordersByUsersB = (
    orders[orders['group'] == 'B']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersB.columns = ['visitorId', 'orders'] 

sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])), name='orders')],axis=0)

sampleB = pd.concat([ordersByUsersB['orders'],pd.Series(0, index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), name='orders')],axis=0)

print('p-value =','{0:.5f}'.format(stats.mannwhitneyu(sampleA, sampleB)[1]))
print('Относительное различие между группами В и А =', '{0:.3f}'.format(sampleB.mean()/sampleA.mean()-1)) 

<div class="alert alert-block alert-warning">
    
**Комментарий от ревьюера**

Этот код можно сократить: нам не нужно собирать кумулятивные данные для проверки гипотез, потому что это уже сделано выше.

</div>

**Вывод:**  
p-value = 0.01679 - это меньше 0.05, поэтому нулевую гипотезу отвергаем. Анализ "сырых" данных показывает, что в среднем количестве заказов между группами есть статистически значимые различия.  
Относительный прирост среднего группы В к группе А составляет 13,8%.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**

Верно, отвергаем нулевую гипотезу: между группами есть статистически значимое различие.
</div>

~~Проверим нулевую гипотезу: между группами А и В нет статистически значимых различий.  
Принимаем значение alpha = 0.05.  
Применим метод Уилкоксона-Манна-Уитни для двух выборок с вероятными выбросами.~~

Проверим **нулевую гипотезу**:  
Статистически значимых различий в среднем чеке заказа по посетителям между группами по "сырым" данным нет.  
и **альтернативную гипотезу**:  
Статистически значимые различия в среднем чеке заказа по посетителям между группами по "сырым" данным есть.  
Принимаем значение alpha = 0.05.  
Применим метод Уилкоксона-Манна-Уитни для двух выборок с вероятными выбросами.

In [ ]:
# посчитаем статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным
print('p-value =','{0:.3f}'.format(stats.mannwhitneyu(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'])[1]))
print('Относительное различие между группами В и А =','{0:.3f}'.format(orders[orders['group']=='B']['revenue'].mean()/orders[orders['group']=='A']['revenue'].mean()-1)) 


<div class="alert alert-block alert-warning">
    
**Комментарий от ревьюера**

Будет удобно, если ты добавишь названия обоих выводимых чисел для всех расчетов. Так их нельзя будет перепутать.

</div>

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера 2**

Когда числа подписаны, вывод сделать проще.
</div>

**Вывод:**  
p-value = 0.729 - это больше 0.05, нулевую гипотезу не отвергаем. Статистически значимых отличий в среднем чеке между группами нет.  
Относительное различие между группами составляет 25,9%.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**

Да, по этому показателю между группами нет статистически значимых различий.
</div>

~~Проверим нулевую гипотезу: между группами А и В нет статистически значимых различий.  
Принимаем значение alpha = 0.05.  
Применим метод Уилкоксона-Манна-Уитни для двух выборок с вероятными выбросами.~~

Проверим **нулевую гипотезу**:  
Статистически значимых различий в среднем количестве заказов на посетителя между группами по "очищенным" данным нет.  
и **альтернативную гипотезу**:  
Статистически значимые различия в среднем количестве заказов на посетителя между группами по "очищенным" данным есть.  
Принимаем значение alpha = 0.05.  
Применим метод Уилкоксона-Манна-Уитни для двух выборок с вероятными выбросами.

In [ ]:
#зададим предельные кол-во заказов и сумму покупки для формирования списка аномальных пользователей
limit_orders = np.percentile(ordersByUsers['orders'], 95) # принимаем границу выбросов по количеству заказов
limit_revenue = np.percentile(transactionBySumm['revenue'], 95) # примимаем границу выбросов по сумме заказов

In [ ]:
# посчитаем статистическую значимость различий в среднем количестве заказов на посетителя между группами по «очищенным» данным

usersWithManyOrders = pd.concat(
    [
        ordersByUsersA[ordersByUsersA['orders'] > limit_orders]['visitorId'],
        ordersByUsersB[ordersByUsersB['orders'] > limit_orders]['visitorId'],
    ],
    axis=0,
)
usersWithExpensiveOrders = orders[orders['revenue'] > limit_revenue]['visitorId']
abnormalUsers = (
    pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis=0)
    .drop_duplicates()
    .sort_values()
)
#4
sampleAFiltered = pd.concat(
    [
        ordersByUsersA[
            np.logical_not(ordersByUsersA['visitorId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

sampleBFiltered = pd.concat(
    [
        ordersByUsersB[
            np.logical_not(ordersByUsersB['visitorId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])
            ),
            name='orders',
        ),
    ],
    axis=0,
)

print('p-value =','{0:.5f}'.format(stats.mannwhitneyu(sampleAFiltered, sampleBFiltered)[1]))
print('Относительное различие между группами В и А =','{0:.3f}'.format(sampleBFiltered.mean()/sampleAFiltered.mean()-1))

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера**

Лучше избегать ручного ввода значений при фильтрации: можно использовать непосредственно результат расчета выбранного перцентиля. 
    
Для этого вписанные вручную числа надо заменить на такие же расчеты, как ты использовал для границы определения аномальных заказов, только там ты указываешь список перцентилей, а здесь нам нужен один.
    
Кроме того выбор границы выбросов в 200 000 получается не совсем удобным. Если сейчас тебя спросят, по какому перцентилю ты провел границу выбросов, ты не сможешь точно ответить. Лучше определять выбросы по тому перцентилю, показатели по которому были рассчитаны. Или рассчитать перцентиль для того количества заказов, которое ты выбираешь как границу выбросов.

</div>

<div class="alert alert-info">
Не понимаю каким образом это реализовать, добавляю переменные.
</div>

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера 2**

Здесь суть в том, чтобы не менять вписанные вручную числа, если поменяются данные. Например, вместо числа 2 можно добавить автоматический расчет перцентиля так:
    
    np.percentile(ordersByUsers['orders'], 95)
</div>

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера 3**

Теперь все правильно, супер.
</div>

---------------------------

~~**Вывод:**  
Как и в случае с "сырыми" данными, статистическая значимость достигнута.  
Группа В превышает А на 16,6%.~~

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера**

Здесь и для следующего вопроса надо добавить в вывод, можем ли мы отвергнуть нулевую гипотезу.

</div>

<div class="alert alert-block alert-danger">
    
**Комментарий от ревьюера 2**

В вывод все еще надо добавить, можем ли мы отвергнуть нулевую гипотезу.

</div>

<div class="alert alert-info">
Исправляю
</div>

p-value = 0.01304 - это меньше 0.05, поэтому нулевую гипотезу отвергаем. Анализ "очищенных" данных показывает, что в среднем количестве заказов на посетителя между группами есть статистически значимые различия.
Относительный прирост среднего группы В к группе А составляет 17,3%.

~~Проверим нулевую гипотезу: между группами А и В нет статистически значимых различий.  
Принимаем значение alpha = 0.05.  
Применим метод Уилкоксона-Манна-Уитни для двух выборок с вероятными выбросами.~~

Проверим **нулевую гипотезу**:  
Статистически значимых различий в среднем чеке заказа между группами по "очищенным" данным нет.  
и **альтернативную гипотезу**:  
Статистически значимые различия в среднем чеке заказа между группами по "очищенным" данным есть.  
Принимаем значение alpha = 0.05.  
Применим метод Уилкоксона-Манна-Уитни для двух выборок с вероятными выбросами.

In [ ]:
# посчитаем статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным. 
print('p-value =',
    '{0:.3f}'.format(
        stats.mannwhitneyu(
            orders[
                np.logical_and(
                    orders['group'] == 'A',
                    np.logical_not(orders['visitorId'].isin(abnormalUsers)),
                )
            ]['revenue'],
            orders[
                np.logical_and(
                    orders['group'] == 'B',
                    np.logical_not(orders['visitorId'].isin(abnormalUsers)),
                )
            ]['revenue'],
        )[1]
    )
)

print('Относительное различие между группами В и А ='
    "{0:.3f}".format(
        orders[
            np.logical_and(
                orders['group'] == 'B',
                np.logical_not(orders['visitorId'].isin(abnormalUsers)),
            )
        ]['revenue'].mean()
        / orders[
            np.logical_and(
                orders['group'] == 'A',
                np.logical_not(orders['visitorId'].isin(abnormalUsers)),
            )
        ]['revenue'].mean()
        - 1
    )
)

**Вывод:**  
После исключения выбросов p-value уменьшился и разница между средними чеками по группам сократилась до 2%.

p-value = 0.738 - это больше 0.05, нулевую гипотезу не отвергаем. Статистически значимых отличий в среднем чеке между группами нет.
Относительное различие между группами В и А составляет -2 %.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**

Да, скорее всего на разницу повлияли выбросы.
</div>

**ОБЩИЙ ВЫВОД:**  
1. Нет статистически значимого различия по среднему количеству заказов между группами ни по «сырым», ни по данным после фильтрации аномалий;
2. Есть статистически значимое различие по среднему чеку между группами по «сырым» и по данным после фильтрации аномалий;
3. График различия среднего количества заказов между группами сообщает, что результаты группы B лучше группы A и нет значительной тенденции к улучшению:
4. График различия среднего чека колеблется: он-то и позволил найти аномалии. Сделать из этого графика определённые выводы нельзя.  
Исходя из обнаруженных фактов, тест следует остановить и признать его успешным. Победителем стоит признать группу В, в которой среднее количество заказов значительно выше, чем в группе А.

<div class="alert alert-block alert-success">
    
**Комментарий от ревьюера**
    
Верно, данных для принятия решения у нас уже достаточно. 
</div>

<div class="alert alert-block alert-info">

<b>Итоговый комментарий ревьюера:</b>

Благодарю тебя за выполнение проекта. Вижу ты постарался, когда работал над ним. Однако, в проекте есть несколько замечаний, которые надо исправить:

* добавить проверку на дубликаты;
* дополнить оформление графиков;
* выбрать границу выбросов в разделе с точечными графиками;
* оформить статистические расчеты;
* не использовать ручной ввод при фильтрации.
       
Жду проект после доработки. Уверена, ты справишься. 💪
</div>

<div class="alert alert-block alert-info">

<b>Итоговый комментарий ревьюера 2:</b>

Замечания, которые осталось исправить:

* выбрать границу выбросов в разделе с точечными графиками;
* оформить статистические расчеты;
* не использовать ручной ввод при фильтрации.
       
</div>

<div class="alert alert-info">
Кажется ничего не забыл
</div>